# Проектная работа 3 Курс 2021/2022
# Идентификация фунции фитнеса методами машинного обучения

# Этап 0 | Постановка задачи | Ожидаемый результат

Цель проекта: Восстановление функции фитнеса, с помощью различных методов машинного обучения

План работы:
- Теоретический модуль
   1. Для начала стоит изучить доступную литературу по данному предмету (список находится в конце Этапа 0)
   2. Изучить базовые возможности и синтаксис языка программирования Python
   3. Создание и исследование математической модели
   4. Ознакомление с возможностями Jupiter Notebook
- Практический модуль
   1. Создание обучающей выборки
   2. Обучение классификаторов
   3. Правка багов и аномалий (тестирование и исправление ошибок)
   4. Написание финального отчета по окончаю проекта

Краткое изложение всей теоретической базы (*За более подробными данными в список литературы*)

Функция фитнеса - это функция биологической приспособленности
- Благодаря выявлению функции фитнеса есть возможность найти оптимальное решение
- Следовательно найти оптимальную стратегию поведения

Фитнес всегда определятеся по разному, но по всем (**моим**) наблюдениям это какая-то функция, которая зависит от различных параметров. Эти параметры (*Макропараметры*) в данной проектной работе представляют из себя описание окружающей среды, количество хищников, естественную смертность и тд. (*Более подробно в Этапе 1)

> -Почему объектом изучения является Зоопланктон?

> -Ну потому что это хорошо изученные организмы, + они перемещаются по всеми известной синусоиде (**Ни в коем случае не путать с косинусоидой о которой как оказалось никто не знает**)

> -Кстати говоря их перемещения носят отдельной научное название - суточные вертикальные миграции (СВМ) или на english (DVM - Diel vertical migration)
> -Ну а еще если переводить это в математику, то окажется, что выходит довольно простая система, с большим количесвом эмпирических данных

> -Я понял, можешь больше не продолжать, душнила.

Ну и после небольших бед с б**кой можно поговорить об обучающей выборке:
- Выборка - конечный отбор чего-то с известной принадлежностью к одному из двух множеств
- И в работе необходимо будет создать обучающую выборку, в которой представлены признаки и принадлежность к определенному классу

В проектной работе в качестве стратегий будут две синусоиды (**НЕ КОСИНУСОИДЫ**), где одна относится к молодым особям, ну то есть к джунам, а другая к взрослым, ну или к сеньорам. Ну и касаемо ранжирования стоит сказать что в данном случае у нас будут сравниваться i-ый элемент с со всеми j-ими.

В проекте мы реализуем 2 главных метода классификации:
- Метод опорных векторов или на english мове (SVM)
- Фишеровский дискриминант или на мове (LDA)
- *Бонусом можно обучить нейронную сеть, но это если будет желание)

Ну и в итоге планируется получить как можно более точные и правдоподобные результаты (*по крайней мере чтобы не оказалось так, что хищники увеличили популяцию зооплантончика нашего ненаглядного*)

Чорт, чуть не забыл про литературу нашу любимую:

- [*Exploring Evolutionary Fitness in Biological Systems Using Machine Learning Methods*](https://www.mdpi.com/1099-4300/23/1/35/htm)
- [*Лекции от Вышки по МО*](https://github.com/esokolov/ml-course-hse)
- [*Почему только 2, а потому что все легко гуглится мы же в 21 веке живем чувак) Это кстати тоже ссылка можешь перейти*](https://www.youtube.com/watch?v=dQw4w9WgXcQ)

# Этап 1 | Обучающая выборка | Ранжирование

## 1. Построение обучающей выборки (Теория)

Пусть имеют место параметризации функций окружающей среды:

- $E(x)$ - пространственное распределение пищи по вертикальной координате $x$

- $S(x)$ - пространственное распределение хищников по вертикальной координате $x$

- $S(t)$ - активность хищников

- $G(x)$ - совокупность дополнительных факторов окружающей среды, типо перепадов температур, радиция и тд.

И нам известны параметры $D$ - максимальная глубина и $D_0$ - оптимальная глубина. А так же $\sigma_1 = 1$, $\sigma_2 = 1$
$$
\begin{cases}
    E = \sigma_1 (x + D)\\
    S_x = \sigma_2 (x + D)
\end{cases}
-D < x < 0
$$
$$
\begin{cases}
    G = (x + D_0)^2\\
    S_{\tau} = cos(2\pi\tau) + 1
\end{cases}
0 < \tau < 1
$$
Возьмем в качестве $v_i$ стратегии две стадии жизнии зоопланктона: молодых и взрослых особей
$$
x_{J} = A_{J} + B_{J}cos(2\pi\tau)\ \ \ \ \ \ x_{A} = A_{A} + B_{A}cos(2\pi\tau)
$$

Где мы должны ограничения для параметров A и B такие что: $ -D < A < 0$, $|B| < min(A + D, -A)$

Ключевыми макропараметрами являются:

$ M_1 $ и $ М_5 $ - количество потребляемой пищи в сутки

$ M_2 $ и $ М_6 $ - суточная смертность от хищников

$ M_3 $ и $ М_7 $ - метаболическая стоимость вертикальной миграции

$ M_4 $ и $ М_8 $ - смертность зоопланктона от неблагоприятных условий среды

        Для молодых особей:
$$
M_1(v_i) = \int_0^1 E(x_{Ji}(\tau))d\tau\ \ \ \ \ \ M_2(v_i) = \int_0^1 S_{\tau}(\tau)S_x(x_{Ji}(\tau))d\tau\ \ \ \ \ \ M_3(v_i) = -\int_0^1 (x'_{Ji}(\tau))d\tau\ \ \ \ \ \ M_4(v_i) = -\int_0^1 G(x_{Ji}(\tau))d\tau
$$
        Для взрослых особей:
$$
M_5(v_i) = \int_0^1 E(x_{Ai}(\tau))d\tau\ \ \ \ \ \ M_6(v_i) = \int_0^1 S_{\tau}(\tau)S_x(x_{Ai}(\tau))d\tau\ \ \ \ \ \ M_7(v_i) = -\int_0^1 (x'_{Ai}(\tau))d\tau\ \ \ \ \ \ M_8(v_i) = -\int_0^1 G(x_{Ai}(\tau))d\tau
$$

Известная функция фитнеса: $ J = -s-p-q+\sqrt[2]{4rp+(p+q-s)^2} $

С коэффициентами:

$ r_i = \alpha_{A} M_5 + \beta_{A} M_7 + \delta_{A} M_8 $


$ s_i = \gamma_{A}M_6 $


$ p_i = \alpha_{J}M_1 + \beta_{J} M_3 + \delta_{J} M_4 $


$ q_i = \gamma_{J}M_2 $

## 2. План действий

### 2.1 Прямой ход работы
Мы генериеруем ключевые параметры А и В стратегий $v_i$ для двух стадий зоопланктона: молодых и взрослых особей. 

Далее мы с помощью А и В уже находим макропараметры $M_1,...,M_8$ и уже зная их мы можем с помощью имеющихся эмпирических данных посчитать значения функции фитнеса для всех стратегий. 

Следующим шагом нам предстоит воспользоваться сравнением пар получившихся значений фитнеса, чтобы разбить их на два базовых класс и уже эти классы мы предъявим нашему классификатору для непосредственного ранжирования с помощью метода опорных векторов или линейного дискриминанта Фишера 

### 2.2 Программный ход работы

Тут будет описание работы программы но чуточку позже, ближе к концу

## 3. Реализация

In [32]:
# Эмпирические данные 
alpha_j = 0.0016
alpha_a = 0.006
betta_j = 0.0000007
betta_a = 0.000000075
gamma_j = 0.00008
gamma_a = 0.004
delta_j = 0.000016
delta_a = 0.00006
sigma1 = 1
sigma2 = 1

D = 120
D0 = 70  

Nmax = 40

In [33]:
import random as rn
import pandas as pd
# Генерация параметров для стратегий
# Запись параметров  
def rand_AB(Nmax):
   A_j, B_j, A_a, B_a = [], [], [], []
   for i in range(0, Nmax):
      Aj = round(rn.uniform(-D, 0), 4)
      Bj = round(rn.uniform(-min(Aj + D, -Aj), min(Aj + D, -Aj)), 4)
      A_j.append(Aj)
      B_j.append(Bj)

      Aa = round(rn.uniform(-D, 0), 4)
      Ba = round(rn.uniform(-min(Aa + D, -Aa), min(Aa + D, -Aa)), 4)
      A_a.append(Aa)
      B_a.append(Ba)
   data_strat = {'A_J': A_j, 'B_J': B_j, 'A_A': A_a, 'B_A': B_a}
   data = pd.DataFrame(data = data_strat)
   data.to_csv("data_strat.csv", index=False)
   return data
data_strat = rand_AB(Nmax)
data_strat

,A_J,B_J,A_A,B_A
0,-60.1346,-32.6629,-35.4272,29.5460
1,-25.8131,-21.9121,-48.2412,37.9159
2,-98.2976,-15.9158,-56.2180,-36.5711
3,-33.2746,10.0663,-66.3014,12.5091
4,-54.9623,-19.9078,-20.2402,-8.4061
5,-21.9273,4.7861,-47.8898,-16.8330
6,-26.0786,-11.8933,-30.6271,13.4434
7,-14.5706,10.6422,-87.3598,29.0027
8,-47.0833,11.2081,-10.9472,-8.6473
9,-99.5470,2.8772,-111.9435,-8.0189


In [34]:
########################################### Сделать красивеее, блять эта хуета наверняка выглядит компактнее ####################
import pandas as pd
import numpy as np
# Подсчет макропараметров
def macroparams(data_strat):
   Aj = data_strat['A_J'].tolist()
   Bj = data_strat['B_J'].tolist()
   Aa = data_strat['A_A'].tolist()
   Ba = data_strat['B_A'].tolist()
   M1, M2, M3, M4, M5, M6, M7, M8 = [], [], [], [], [], [], [], []
   for i in range(0, Nmax):
      M1.append(round(sigma1*(Aj[i] + D), 5))
      M2.append(round(-sigma2*(Aj[i] + D + Bj[i]/2), 5))
      M3.append(round(-2*(np.pi*Bj[i])**2, 5))
      M4.append(round(-((Aj[i] + D0)**2 + (Bj[i]**2)/2), 5))
      M5.append(round(sigma1*(Aa[i] + D), 5))
      M6.append(round(-sigma2*(Aa[i] + D + Ba[i]/2), 5))
      M7.append(round(-2*(np.pi*Ba[i])**2, 5))
      M8.append(round(-((Aa[i] + D0)**2 + (Ba[i]**2)/2), 5))

   M1_n, M2_n, M3_n, M4_n, M5_n, M6_n, M7_n, M8_n = [], [], [], [], [], [], [], []
   M1_n1, M2_n1, M3_n1, M4_n1, M5_n1, M6_n1, M7_n1, M8_n1 = [], [], [], [], [], [], [], []
   for i in range(0, Nmax):
      M1_n1.append(abs(M1[i]))
      M2_n1.append(abs(M2[i]))
      M3_n1.append(abs(M3[i]))
      M4_n1.append(abs(M4[i]))
      M5_n1.append(abs(M5[i]))      
      M6_n1.append(abs(M6[i]))
      M7_n1.append(abs(M7[i]))
      M8_n1.append(abs(M8[i]))
   
   for i in range(0, Nmax):
      M1_n.append(M1[i]/max(M1_n1))
      M2_n.append(M2[i]/max(M2_n1))
      M3_n.append(M3[i]/max(M3_n1))
      M4_n.append(M4[i]/max(M4_n1))
      M5_n.append(M5[i]/max(M5_n1))      
      M6_n.append(M6[i]/max(M6_n1))
      M7_n.append(M7[i]/max(M7_n1))
      M8_n.append(M8[i]/max(M8_n1))
   data_macro = {'M1': M1_n, 'M2': M2_n, 'M3': M3_n, 'M4': M4_n, 'M5': M5_n, 'M6': M6_n, 'M7': M7_n, 'M8': M8_n}
   data = pd.DataFrame(data = data_macro)
   data.to_csv("data_macro.csv", index=False)
   return M1_n, M2_n, M3_n, M4_n, M5_n, M6_n, M7_n, M8_n

In [35]:
import pandas as pd
# Считаем фитнесс
def fitness(Nmax):
   M1, M2, M3, M4, M5, M6, M7, M8 = macroparams(data_strat)
   J = []
   r, s, p, q = 0, 0, 0, 0
   for i in range(0, Nmax):
      r = alpha_a*M5[i] + betta_a*M7[i] + delta_a*M8[i]
      s = gamma_a*M6[i]
      p = alpha_j*M1[i] + betta_j*M3[i] + delta_j*M4[i]
      q = gamma_j*M2[i]
      if ((4*r*p + (p + q - s)**2) < 0):
         J.append(0)
      else:
         j = -s - p - q + np.sqrt(4*r*p + (p + q - s)**2)
         J.append(j)
   data = pd.read_csv('data_macro.csv')
   data.insert(0, 'J', J)
   return data

data_fit_macro = fitness(Nmax)
data_fit_macro

,J,M1,M2,M3,M4,M5,M6,M7,M8
0,0.008182,0.521488,-0.371240,-0.408414,-0.149793,0.721620,-0.839620,-0.324151,-0.361155
1,0.007959,0.820463,-0.709760,-0.183806,-0.520687,0.612284,-0.766691,-0.533818,-0.263879
2,0.003937,0.189050,-0.117208,-0.096972,-0.220241,0.544222,-0.384513,-0.496623,-0.190047
3,0.005709,0.755466,-0.782480,-0.038791,-0.332334,0.458185,-0.506693,-0.058104,-0.020344
4,0.008292,0.566544,-0.469733,-0.151718,-0.100761,0.851203,-0.807596,-0.026239,-0.555837
5,0.006508,0.854312,-0.856732,-0.008769,-0.551532,0.615282,-0.538306,-0.105214,-0.139555
6,0.008630,0.818150,-0.750214,-0.054150,-0.474916,0.762577,-0.812142,-0.067107,-0.363108
7,0.004518,0.918396,-0.944436,-0.043357,-0.743086,0.278503,-0.398416,-0.312340,-0.159786
8,0.009116,0.635178,-0.669594,-0.048090,-0.139635,0.930496,-0.885117,-0.027766,-0.780098
9,0.000645,0.178166,-0.186683,-0.003169,-0.208309,0.068742,-0.034204,-0.023877,-0.396489


In [39]:
def classificator(differ):
   if differ[0] < 0:
      return -1
   else:
      return 1
      
def data_class():
   data_fit_macro = fitness(Nmax)
   target = []
   for i in range(0, Nmax):
      for j in range(i + 1, Nmax):
         differ_ = data_fit_macro.loc[i] - data_fit_macro.loc[j]
         target_ = classificator(differ_)
         target.append(differ_.append(pd.Series(target_, index =['target'])))
   data = pd.DataFrame(columns=['J', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'target'], data = target)
   data.to_csv('data_vyborka.csv', index=False)
   return data
data = data_class()
data


,J,M1,M2,M3,M4,M5,M6,M7,M8,target
0,0.000223,-0.298975,0.338519,-0.224608,0.370895,0.109336,-0.072928,0.209667,-0.097276,1.0
1,0.004244,0.332438,-0.254033,-0.311442,0.070449,0.177398,-0.455107,0.172471,-0.171108,1.0
2,0.002473,-0.233978,0.411240,-0.369623,0.182542,0.263435,-0.332926,-0.266048,-0.340811,1.0
3,-0.000111,-0.045056,0.098493,-0.256696,-0.049032,-0.129583,-0.032023,-0.297913,0.194682,-1.0
4,0.001673,-0.332824,0.485492,-0.399645,0.401740,0.106337,-0.301313,-0.218937,-0.221599,1.0
...,...,...,...,...,...,...,...,...,...,...
775,-0.005393,-0.616590,0.535640,0.241401,0.056935,-0.371010,0.521284,0.910795,0.035982,-1.0
776,-0.003489,-0.042482,0.049765,-0.015931,-0.082506,-0.362451,0.403882,0.164107,-0.190551,-1.0
777,-0.001592,-0.539393,0.424489,0.290191,0.204630,0.005763,0.089660,0.680645,0.199907,-1.0
778,0.000312,0.034715,-0.061387,0.032858,0.065189,0.014323,-0.027742,-0.066042,-0.026625,1.0


In [51]:
""" X=data_class().loc[:,'M1':'M8']
y=data_class().loc[:,'target']

import matplotlib.pyplot as plt
%matplotlib inline

X1 = X[y==1]
X0 = X[y==-1]

for i in range(len(X.columns)):
    for j in range(i + 1, len(X.columns)):
        x=np.linspace(-1, 1)
        plt.figure(figsize=(7, 7))
        plt.scatter(x = X1[X.columns[i]], y=X1[X.columns[j]], marker='.')
        plt.scatter(x = X0[X.columns[i]], y=X0[X.columns[j]], marker='x')
        plt.xlim(-1,)
        plt.ylim(-1,1)
        plt.legend()
        plt.xlabel(X.columns[i])
        plt.ylabel(X.columns[j])
        plt.grid()
        plt.show() """

" X=data_class().loc[:,'M1':'M8']\ny=data_class().loc[:,'target']\n\nimport matplotlib.pyplot as plt\n%matplotlib inline\n\nX1 = X[y==1]\nX0 = X[y==-1]\n\nfor i in range(len(X.columns)):\n    for j in range(i + 1, len(X.columns)):\n        x=np.linspace(-1, 1)\n        plt.figure(figsize=(7, 7))\n        plt.scatter(x = X1[X.columns[i]], y=X1[X.columns[j]], marker='.')\n        plt.scatter(x = X0[X.columns[i]], y=X0[X.columns[j]], marker='x')\n        plt.xlim(-1,)\n        plt.ylim(-1,1)\n        plt.legend()\n        plt.xlabel(X.columns[i])\n        plt.ylabel(X.columns[j])\n        plt.grid()\n        plt.show() "

In [106]:
import pandas as pd
from sklearn.model_selection import train_test_split

X = data.iloc[:, 1:-1].values 
y = data['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1) # 1 и 27 - для максимально точных результатов

In [107]:
from sklearn.svm import SVC
SVC_model = SVC(kernel= 'linear')
SVC_model.fit(X_train, y_train)
SVC_prediction = SVC_model.predict(X_test) 

In [108]:
from sklearn import discriminant_analysis
lda_model = discriminant_analysis.LinearDiscriminantAnalysis()
lda_model.fit(X_train, y_train)
LDA_prediction = lda_model.predict(X_test) 

In [109]:
from sklearn.metrics import accuracy_score
print(accuracy_score(SVC_prediction, y_test))
print(accuracy_score(LDA_prediction, y_test)) 

0.9871794871794872
0.9807692307692307
